# Imports & Globals

In [65]:
import numpy as np
import pandas as pd
import pickle
from yaml import safe_load
from difflib import get_close_matches
from rapidfuzz.distance import Levenshtein

import re
import string
from typing import Literal

#import matplotlib.pyplot as plt
#import seaborn as sns
# %matplotlib inline

In [66]:
import tensorflow as tf
#@title Versions:
print("tf.version: ", tf.version.VERSION)
print("tf.keras.version: ", tf.keras.__version__)

tf.version:  2.9.1
tf.keras.version:  2.9.0


In [67]:
# from tensorflow.python.ops.numpy_ops import np_config
# np_config.enable_numpy_behavior()
# tf.enable_eager_execution()

In [68]:
# Check that GPU is available: cf. https://colab.research.google.com/notebooks/gpu.ipynb
assert(tf.test.gpu_device_name())

tf.keras.backend.clear_session()
tf.config.optimizer.set_jit(True) # Enable XLA.

2022-08-29 14:51:41.880374: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-29 14:51:41.880566: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-29 14:51:41.880677: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-29 14:51:41.880821: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-29 14:51:41.880932: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from S

In [69]:
tf.executing_eagerly()

True

In [70]:
with open('LSTM65.yaml', 'r') as file:
    model_config = safe_load(file)

In [71]:
#Maximum sequence length including padding
global MAXLEN
MAXLEN = model_config['MAXLEN']

In [72]:
global encode_dict 
encode_dict = {l:i for i,l in enumerate(string.ascii_uppercase + " ", 1)}

## Data Processing Functions

In [73]:
def add_noise(w: str, percent: float = 0.1) -> str:
    ''' Adds a specified proportion of noise to a string.
    
    Expects a string and a number stating the percent of noise to add to this string.
    The string is modified by editing, deleting, or adding characters in/to the string.
    The modification to perform is determined randomly by generating a random number from an uniform distribution [0,1].
    If the number is < 1/3 edit one position with new random character or space.
    If the number is < 2/3 delete one position.
    Finally, if the number is > 2/3 add one random character or space.
    
    In order to retain the length of the sequence compliant with the maximum sequence length,
    additional processing has been added such that sequences that reach the maximum sequence length
    can only be modified by removing or swapping characters.
    
    Parameters
    ----------
    w : str
        The string to add noise to.
    
    percent: float, defaults to 10% if not specified
        Percentange representing the proportion of noise to add to the string.
        
    Returns
    -------
    w : str
        Modified string with noise added.
    '''  
    positions = random.choices(range(len(w)), k=int(percent*len(w)))
#     print("Adding noise to", int(percent*len(w)), "% of the string")
    for p in positions:
        r = random.uniform(0,1)
        if len(w) < MAXLEN:
            # if <1/3 edit one position with new random character, # else if <2/3 delete one position, else add one random character 
            if r <= 0.3333: # edit
                w = w[:p] + random.choice(string.ascii_uppercase + " ") + w[p+1:]
            elif r<= 0.6667: # delete
                w = w[:p] + w[p+1:]
            else: # add
                w = w[:p] + random.choice(string.ascii_uppercase + " ") + w[p:]
        else:
            if r <= 0.5: # edit
                w = w[:p] + random.choice(string.ascii_uppercase + " ") + w[p+1:]
            else: # delete
                w = w[:p] + w[p+1:]
            
    return w

In [74]:
def clean(text: str) -> str:
    '''Removes all the non-ascii and special characters from a string and returns the string's alphabetichal characters with spaces.
    
    Expects a string to be cleaned and removes all the non-ascii and special characters. 
    This is done by applying a substitution to regex matches
    Returns the cleaned string containing uppercased versions of the characters.
    
    Parameters
    ----------
    text : str
        
    Returns
    -------
    text : str
    '''
    regex = re.compile('[^a-zA-Z ]')
    r = regex.sub('', text)
    result = re.sub(' +', ' ', r)
    result = result.strip()
    return result.upper()

In [75]:
def encode(x: string) -> [int]:
    '''Applies the encoding function to a given value.
    
    Returns these string enconded into an array containing an integer mapping to each character and space (1-66) separately.
    
    Parameters
    ----------
    x : string
        
      
    Returns
    -------
    x : [int]
        Returns the encoded string.

    '''
    return list(map(encode_dict.get, x))

In [76]:
def padding(x: [int], maxlen: int = MAXLEN) -> [int]:
    '''Applies the padding function to the encoded sequence.
    
    Returns the enconded Series padded.
    
    Parameters
    ----------
    x : [int]
       Encoded character sequence.
      
    Returns
    -------
    x : [int]
        Returns the padded encoded character sequence.
    '''
    return x + ([0]* (maxlen-len(x)))

In [77]:
def preprocessInput(filename: str, maxlen: int = MAXLEN, **kwargs) -> pd.DataFrame:
    '''Preprocess CSV file into a Pandas DataFrame.
    
    Expects the file name or path of a csv file with named columns containing strings representing product names.
    It then removes the sequences with length greater than the maximun sequence length, cleans the sequences and
    uppercases them, and it finally drops any duplicates that might have arrisen from this processing.
    Returns a Pandas Dataframe containing unique cleaned and uppercased versions of the strings on each cell.
    
    Parameters
    ----------
    filename : str
        
    Returns
    -------
    df : Pandas DataFrame
    '''  
    df = pd.read_csv(filename, **kwargs)
    print(df.info())
    
    
    print("Processing file: ----------------------------------------")
    
    print("Renaming colums:")
    print("\tCurrent names: {}".format(df.columns))
    cols = df.columns.size
    match cols:
        case 1: 
            df.columns = ["x"]
        case 2:
            df.columns = ["x", "y"]
    print("\tNew names: {}".format(df.columns))
    
    original_count = len(df.index)
    print("Dropping row with empty cells:")
    df.dropna(subset=df.columns, inplace=True)
    new_count = len(df.index)
    print("\tDropped", original_count - new_count, "rows with empty cells.")
    
    original_count = len(df.index)
    print("Dropping sequences longer than the maxlen of {}:".format(maxlen))
    for column in df.columns:
        df.drop(df[df[column].apply(len).gt(maxlen)].index, inplace = True)
    new_count = len(df.index)
    print("\tDropped", original_count - new_count, "that exceeded the maximum sequence length.")
    
    print("\tCleaning string sequences.")
    df = df.applymap(clean)
    
    print("\tUppercasing string sequences.")
    df = df.applymap(lambda x: str.upper(x))
    
    print("Dropping duplicate sequences:")
    original_count = len(df.index)
    df.drop_duplicates(ignore_index=True, inplace=True)
    new_count = len(df.index)
    print("\tDropped", original_count - new_count, "duplicate sequences.")
    
    print("Done processing: ---------------------------------------")
    print(df.info())
    return df

In [78]:
def encode_pad_tag(df: pd.DataFrame, match: Literal[0,1], distance: Literal[0,1], maxlen: int = MAXLEN) -> pd.DataFrame:
    '''It encodes, pads and tags the preprocessed sequences in a Pandas DataFrame.
    
    Expects a pandas dataframe with cleaned and uppercased sequences. It processes the 
    the DataFrame by creating an additional 'Processed_' + current column name for each
    of the columns in the data frame, where each of the sequences in the column get 
    transformed from a string sequence to an encoded sequence and then transformed again 
    by padding the encoded sequences up to the maximun sequence length by 0's as needed. 
    The tag component 1 
    Finally, this function returns this dataframe with both the original and processed columns.
    
    Parameters
    ----------
    df : pd.DataFrame
        DataFrame containing sequences.
    
    match: Literal[0 | 1]
        Tag indicating wether sequences match 1 indicates 'yes' and 0 indicates 'no'.
        
    distance: Literal[0 | 1]
        Distance between sequences, 1 indicates 'far' and 0 indicates 'close'. 
        
    maxlen: int
        Dafault to global MAXLEN value. It's used to determine the ammount of padding to add
        to sequences smaller than maxlen.
        
    Returns
    -------
    df : Pandas DataFrame
        A copy of the origininal DataFrame with the processed sequences added as new columns.
    '''  
    print("Encoding and Padding: ----------------------------------")
    for column in df.columns:
        print("\tProcessing {}".format(column))
        df["Processed_" + column] = df[column].apply(lambda string: list(map(encode_dict.get, string))).transform(lambda x: x + ([0]* (maxlen-len(x))))
    print("Tagging: -----------------------------------------------")
    df["Match"] = match
    df["Distance"] = distance
    return df

------------------------------------------------

# Data loading and preprocessing

# Pickled Datasets

In [79]:
dUnique_df = pd.read_pickle("../data/dUnique_df.pkl")
dfneg2 = pd.read_pickle("../data/dfneg2.pkl")
test = pd.read_pickle("../data/test.pkl")
validate = pd.read_pickle("../data/validate.pkl")

--------------------------------

# Build model, load weights and evaluate on test data

In [80]:
class CosineSimilarity(tf.keras.layers.Layer):
    '''Cosine similarity to be calculated as sum(x*y)/(sqrt(sum(x))*sqrt(sum(y))).
    This is achieved through Tensorflow functions to retain performance.
    
    Parameters
    ----------
    vects: tf.TensorArray
    
    Returns
    -------
    cosine_similarity: tf.TensorArray
       The result of the cosine similarity between the vectors.    
    '''
    __name__ = 'CosineSimilarity'
    def __init__(self, **kwargs):
        super(CosineSimilarity, self).__init__()
       
    @tf.function  # The decorator converts `cosine_similarity` into a tensolflow `Function`.
    def call(self, vects: tf.TensorArray) -> tf.TensorArray:
        x, y = vects
        return tf.math.divide(tf.reduce_sum(tf.multiply(x,y), axis=1, keepdims=True), tf.multiply(tf.norm(x, ord=2, axis=1, keepdims=True), tf.norm(y, ord=2, axis=1, keepdims=True)))

    def get_config(self):
        return super(CosineSimilarity, self).get_config()
    
    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [81]:
class CosineDistance(tf.keras.layers.Layer):
    '''Cosine distance to be calculated as 1-(cosine similarity).
    Where cosine similarity equals sum(x*y)/(sqrt(sum(x))*sqrt(sum(y))).
    This is achieved through Tensorflow functions to retain performance.
    
    Parameters
    ----------
    vects: tf.TensorArray
        
    
    Returns
    -------
    cosine_distance: tf.TensorArray
        The result of 1-cosine similarity between the vectors. 
    '''
    __name__ = 'CosineDistance'
    def __init__(self, **kwargs):
        super(CosineDistance, self).__init__()
       
    @tf.function  # The decorator converts `cosine_distance` into a tensorflow `Function`.
    def call(self, vects: tf.TensorArray) -> tf.TensorArray:
        x, y = vects
        return  tf.math.subtract(tf.constant([1.0]), tf.math.divide(tf.reduce_sum(tf.multiply(x,y), axis=1, keepdims=True), tf.multiply(tf.norm(x, ord=2, axis=1, keepdims=True), tf.norm(y, ord=2, axis=1, keepdims=True))))

    def get_config(self):
        return super(CosineDistance, self).get_config()
    
    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [82]:
class ContrastiveLoss(tf.keras.losses.Loss):
    '''Returns a value between 0 and 1 representing the average error of the y_pred vector by comparing it to the y_true.
    '''
    __name__ = 'ContrastiveLoss'
    def __init__(self, margin: tf.float32 = 1.0, **kwargs):
        super(ContrastiveLoss, self).__init__()
        self.margin = tf.constant(margin)
        
    @tf.function  # The decorator converts `loss` into a tensolflow `Function`.
    def call(self, y_true: tf.TensorArray, y_pred: tf.TensorArray) -> tf.Tensor:
        return tf.math.reduce_mean((1 - y_true) * tf.math.square(y_pred) + (y_true) * tf.math.square(tf.math.maximum(self.margin - (y_pred), 0.0)), axis = -1)
    
    def get_config(self):
        config = super(ContrastiveLoss, self).get_config()
        config.update({
            "margin": str(self.margin)
        })
        return config
    
    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [83]:
model = tf.keras.models.load_model(
    './saved_models/{}_extended'.format(model_config["Model_Name"]), 
    custom_objects = {
#         'CosineSimilarity': CosineSimilarity,
        'CosineDistance': CosineDistance,
        'ContrastiveLoss': ContrastiveLoss
    }, 
    compile=True, 
    options=None
)
model.summary()

2022-08-29 14:51:42.764227: W tensorflow/core/common_runtime/graph_constructor.cc:805] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-08-29 14:51:43.039808: W tensorflow/core/common_runtime/graph_constructor.cc:805] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-08-29 14:51:43.214745: W tensorflow/core/common_runtime/graph_constructor.cc:805] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-08-29 14:51:43.223247: W tensorflow/core/common_runtime/graph_constructor.cc:805] Node 'cond' has 5 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-08-29 14:51:43.359231: W tensorflow/core/common_runtime/graph_constructor.cc:805] Node 'cond/while' has 13 outputs but the _ou

2022-08-29 14:51:47.994121: W tensorflow/core/common_runtime/graph_constructor.cc:805] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-08-29 14:51:48.003394: W tensorflow/core/common_runtime/graph_constructor.cc:805] Node 'cond' has 5 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-08-29 14:51:48.231263: W tensorflow/core/common_runtime/graph_constructor.cc:805] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-08-29 14:51:48.240310: W tensorflow/core/common_runtime/graph_constructor.cc:805] Node 'cond' has 5 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-08-29 14:51:48.362406: W tensorflow/core/common_runtime/graph_constructor.cc:805] Node 'cond' has 5 outputs but the _output_shapes at

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 65)]         0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 65)]         0           []                               
                                                                                                  
 model (Functional)             (None, 180)          60108       ['input_2[0][0]',                
                                                                  'input_3[0][0]']                
                                                                                                  
 lambda (Lambda)                (None, 1)            0           ['model[0][0]',            

---------------------------------------------------------------

# Evaluation

## Unique target labels

In [84]:
dUnique_df.head()

,dUnique_label,dUnique_seq_padded
0,ASHWAGANDA,"[1, 19, 8, 23, 1, 7, 1, 14, 4, 1, 0, 0, 0, 0, ..."
1,BUTCHERSBROOM,"[2, 21, 20, 3, 8, 5, 18, 19, 2, 18, 15, 15, 13..."
2,CATSCLAW,"[3, 1, 20, 19, 3, 12, 1, 23, 0, 0, 0, 0, 0, 0,..."
3,CINNAMON,"[3, 9, 14, 14, 1, 13, 15, 14, 0, 0, 0, 0, 0, 0..."
4,FENUGREEK,"[6, 5, 14, 21, 7, 18, 5, 5, 11, 0, 0, 0, 0, 0,..."


In [85]:
dUnique_df.size

1578

## True Positives

In [86]:
matches = encode_pad_tag(preprocessInput('../data/NP_FAERS_mapped_20220215.csv'), match=1, distance=0)
matches.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5358 entries, 0 to 5357
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   FAERS_drug_match  5358 non-null   object
 1   lookup_value      5358 non-null   object
dtypes: object(2)
memory usage: 83.8+ KB
None
Processing file: ----------------------------------------
Renaming colums:
	Current names: Index(['FAERS_drug_match', 'lookup_value'], dtype='object')
	New names: Index(['x', 'y'], dtype='object')
Dropping row with empty cells:
	Dropped 0 rows with empty cells.
Dropping sequences longer than the maxlen of 65:
	Dropped 374 that exceeded the maximum sequence length.
	Cleaning string sequences.
	Uppercasing string sequences.
Dropping duplicate sequences:
	Dropped 482 duplicate sequences.
Done processing: ---------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4502 entries, 0 to 4501
Data columns (total 2 columns):
 #   Column

,x,y,Processed_x,Processed_y,Match,Distance
0,ASHWAGANDHA,ASHWAGANDA,"[1, 19, 8, 23, 1, 7, 1, 14, 4, 8, 1, 0, 0, 0, ...","[1, 19, 8, 23, 1, 7, 1, 14, 4, 1, 0, 0, 0, 0, ...",1,0
1,ASHWAGANDHA EXTRACT,ASHWAGANDA,"[1, 19, 8, 23, 1, 7, 1, 14, 4, 8, 1, 27, 5, 24...","[1, 19, 8, 23, 1, 7, 1, 14, 4, 1, 0, 0, 0, 0, ...",1,0
2,ASHWAGANDHA ROOT EXTRACT,ASHWAGANDA,"[1, 19, 8, 23, 1, 7, 1, 14, 4, 8, 1, 27, 18, 1...","[1, 19, 8, 23, 1, 7, 1, 14, 4, 1, 0, 0, 0, 0, ...",1,0
3,ASHWAGANDHA WITHANIA SOMNIFERA,ASHWAGANDA,"[1, 19, 8, 23, 1, 7, 1, 14, 4, 8, 1, 27, 23, 9...","[1, 19, 8, 23, 1, 7, 1, 14, 4, 1, 0, 0, 0, 0, ...",1,0
4,ASHWAGANDHA WITHANIA SOMNIFERA ROOT,ASHWAGANDA,"[1, 19, 8, 23, 1, 7, 1, 14, 4, 8, 1, 27, 23, 9...","[1, 19, 8, 23, 1, 7, 1, 14, 4, 1, 0, 0, 0, 0, ...",1,0


### ASHWAGANDA

In [87]:
matches.loc[1, "x"]

'ASHWAGANDHA EXTRACT'

In [88]:
predicts = model.predict([np.tile(matches.loc[1, "Processed_x"], (dUnique_df['dUnique_seq_padded'].shape[0],1)), np.stack(dUnique_df['dUnique_seq_padded'])]).astype('float32')
top5 = predicts.flatten().argsort()

25/25 [==============================] - 3s 4ms/step


In [89]:
# Top-5 smalles distances
{
    "rank1": dUnique_df.iloc[top5[0]]['dUnique_label'],
    "rank2": dUnique_df.iloc[top5[1]]['dUnique_label'],
    "rank3": dUnique_df.iloc[top5[2]]['dUnique_label'],
    "rank4": dUnique_df.iloc[top5[3]]['dUnique_label'],
    "rank5": dUnique_df.iloc[top5[4]]['dUnique_label'],
}

{'rank1': 'ASHWAGANDA',
 'rank2': 'SERENOA REPENS',
 'rank3': 'WITHANIA SOMNIFERA',
 'rank4': 'SCRUBPALMETTO',
 'rank5': 'ASPARAGUS SETACEUS'}

### Echinacea

In [90]:
Echinacea = dfneg2[dfneg2["x"].str.contains("ECHINACEA")].head(1)
Echinacea

,x,y,Processed_x,Processed_y,Match,Distance
393,ECHINACEA TEA ECHINACEA PURPUREA,GREEN TEA,"[5, 3, 8, 9, 14, 1, 3, 5, 1, 27, 20, 5, 1, 27,...","[7, 18, 5, 5, 14, 27, 20, 5, 1, 0, 0, 0, 0, 0,...",0,1


In [91]:
i = Echinacea.index.values[0]
clean(dfneg2["x"][i])

'ECHINACEA TEA ECHINACEA PURPUREA'

In [92]:
predicts = model.predict([np.tile(padding(encode(clean(dfneg2["x"][i]))), (dUnique_df['dUnique_seq_padded'].shape[0],1)), np.stack(dUnique_df['dUnique_seq_padded'])]).astype('float32')
top5 = predicts.flatten().argsort()

25/25 [==============================] - 0s 11ms/step


In [93]:
# Top-5 smalles distances
m = max(top5)
{
    "rank1": dUnique_df.iloc[top5[0]]['dUnique_label'],
    "rank2": dUnique_df.iloc[top5[1]]['dUnique_label'],
    "rank3": dUnique_df.iloc[top5[2]]['dUnique_label'],
    "rank4": dUnique_df.iloc[top5[3]]['dUnique_label'],
    "rank5": dUnique_df.iloc[top5[4]]['dUnique_label'],
}

{'rank1': 'CRATAEGUS LAEVIGATA',
 'rank2': 'ECHINACEA',
 'rank3': 'ECHINACEA ANGUSTIFOLIA',
 'rank4': 'WOODLAND HAWTHORN',
 'rank5': 'ECHINACEA PURPUREA'}

### Cranberry

In [94]:
cranberry = dUnique_df[dUnique_df["dUnique_label"].str.contains("CRANBERRY")].head(1)
cranberry

,dUnique_label,dUnique_seq_padded
58,CRANBERRY,"[3, 18, 1, 14, 2, 5, 18, 18, 25, 0, 0, 0, 0, 0..."


In [95]:
i = cranberry.index.values[0]
clean(dUnique_df["dUnique_label"][i])

'CRANBERRY'

In [96]:
predicts = model.predict([np.tile(padding(encode(clean(dUnique_df["dUnique_label"][i]))), (dUnique_df['dUnique_seq_padded'].shape[0],1)), np.stack(dUnique_df['dUnique_seq_padded'])]).astype('float32')
top5 = predicts.flatten().argsort()
# argsort = predicts.flatten().argsort()

25/25 [==============================] - 0s 2ms/step


In [97]:
# Top-5 smalles distances
m = max(top5)
{
    "rank1": dUnique_df.iloc[top5[0]]['dUnique_label'],
    "rank2": dUnique_df.iloc[top5[1]]['dUnique_label'],
    "rank3": dUnique_df.iloc[top5[2]]['dUnique_label'],
    "rank4": dUnique_df.iloc[top5[3]]['dUnique_label'],
    "rank5": dUnique_df.iloc[top5[4]]['dUnique_label'],
}

{'rank1': 'APPLE CIDER VINEGAR',
 'rank2': 'CRANBERRY',
 'rank3': 'VACCINIUM MACROCARPON',
 'rank4': 'BARBERRY',
 'rank5': 'VACCINIUM MYRSINITES'}

## True  Negatives

In [98]:
dfneg2.head()

,x,y,Processed_x,Processed_y,Match,Distance
0,ANUSOLHC BENZYL BENZOATEBISMUTH HYDROXIDE,CINNAMON,"[1, 14, 21, 19, 15, 12, 8, 3, 27, 2, 5, 14, 26...","[3, 9, 14, 14, 1, 13, 15, 14, 0, 0, 0, 0, 0, 0...",0,1
1,ASCABIOL BENZYL BENZOATE,CINNAMON,"[1, 19, 3, 1, 2, 9, 15, 12, 27, 2, 5, 14, 26, ...","[3, 9, 14, 14, 1, 13, 15, 14, 0, 0, 0, 0, 0, 0...",0,1
2,CASSIA,CINNAMON,"[3, 1, 19, 19, 9, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[3, 9, 14, 14, 1, 13, 15, 14, 0, 0, 0, 0, 0, 0...",0,1
3,CASSIA ACUTIFOLIA,CINNAMON,"[3, 1, 19, 19, 9, 1, 27, 1, 3, 21, 20, 9, 6, 1...","[3, 9, 14, 14, 1, 13, 15, 14, 0, 0, 0, 0, 0, 0...",0,1
4,CASSIA ALATA,CINNAMON,"[3, 1, 19, 19, 9, 1, 27, 1, 12, 1, 20, 1, 0, 0...","[3, 9, 14, 14, 1, 13, 15, 14, 0, 0, 0, 0, 0, 0...",0,1


# Evaluating on test data - NP names only

In [99]:
vocab = pd.read_csv('../data/lb_to_common_names.csv')
vocab.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 958 entries, 0 to 957
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   latin_binomial        958 non-null    object
 1   common_name           958 non-null    object
 2   latin_binomial_clean  958 non-null    object
 3   common_name_clean     958 non-null    object
dtypes: object(4)
memory usage: 30.1+ KB


# Create a sample from the test set to evaluate

In [100]:
positive_pairs = test.loc[test["Match"] == 1].sample(n=500)

# Evaluation of drug name predictions
### Find ranks 1-n from the predicted similarities for the test data

In [101]:
def find_ranks(model: tf.keras.Model, df: pd.DataFrame, find_related_rank: bool = False, report_distances: bool = False) -> pd.DataFrame:
    """For each row in the input dataframe, the model is used to predict the top matching Unique Product Names 
    in 'x' against the entry matches any of the 'y' entries.
    This is done at the encoded sequence level for both name all unique drugnames
       
         Parameters
    ----------
    model : tf.keras.Model
        A Keras model based Siamese Network that takes three inputs. 
        Namely, two input sequeces and a third input binary target specifying wether the two sequeces match.
    
    find_related_rank: bool
        A flag indicating wether to compare the top ranked results against the 'y' and it's potential equivalents or not.
        
    report_distances: bool
        A flag indicating return the distance values of the top ranked results against the 'y'.
      
    Returns
    -------
    df : pd.DataFrame
        Returns the padded 'x', 'y', 'rank1', 'rank2', 'rank3', 'rank4', 'rank5' series.
        And additionally the 'exact_rank' and 'equivalent_rank' series and the 'rank1_distance', 'rank2_distance', 
        'rank3_distance', 'rank4_distance', 'rank5_distance' if requested.
    
    """
    
    print("Using column: ", df.columns[0])
    df.assign(rank1="", rank2="", rank3="", rank4="", rank5="")
    
    if report_distances:
        df.assign(rank1_distance=np.Inf, rank2_distance=np.Inf, rank3_distance=np.Inf, rank4_distance=np.Inf, rank5_distance=np.Inf)
 
    if find_related_rank:
        df.assign(exact_rank= np.Inf, equivalent_rank = np.Inf)
    
    for i in df.index:
        predicts = model.predict([np.tile(df.loc[i, "Processed_"+ df.columns[0]], (dUnique_df['dUnique_seq_padded'].shape[0],1)), np.stack(dUnique_df['dUnique_seq_padded'])]).astype('float32')
        argsort = predicts.flatten().argsort()
        # Top-5 smalles distances
        df.at[i, 'rank1'] = dUnique_df['dUnique_label'][argsort[0]]  
        df.at[i, 'rank2'] = dUnique_df['dUnique_label'][argsort[1]]
        df.at[i, 'rank3'] = dUnique_df['dUnique_label'][argsort[2]]
        df.at[i, 'rank4'] = dUnique_df['dUnique_label'][argsort[3]]
        df.at[i, 'rank5'] = dUnique_df['dUnique_label'][argsort[4]]
        
        if report_distances:
            df.at[i, 'rank1_distance'] = predicts[argsort[0]]  
            df.at[i, 'rank2_distance'] = predicts[argsort[1]]
            df.at[i, 'rank3_distance'] = predicts[argsort[2]]
            df.at[i, 'rank4_distance'] = predicts[argsort[3]]
            df.at[i, 'rank5_distance'] = predicts[argsort[4]]

        if find_related_rank:
            # Find the top-5 predicted matches
            lookup_clean = clean(df.at[i , df.columns[1]])
            predicted_rank = df.loc[i, ['rank1', 'rank2', 'rank3', 'rank4', 'rank5']].eq(lookup_clean).to_numpy().nonzero()
            
            # Find the top ranking correct match, if not rank is infinity so that 1/inf ~ 0, for the MRR computation.
            lookup_rank = np.Inf    
            if len(predicted_rank[0]) > 0 :
                lookup_rank = predicted_rank[0][0] + 1
            df.loc[i, "exact_rank"] = lookup_rank

            # Find all the equivalent common names and latin binomials relative to the look up value that would be equaly correct.
            equivalent = np.setdiff1d(vocab[["latin_binomial_clean","common_name_clean"]][(vocab["latin_binomial_clean"] == lookup_clean) | (vocab["common_name_clean"] == lookup_clean)].unstack().unique(), lookup_rank)

            # Find the top ranking correct match 
            related_rank = np.Inf
            if len(equivalent) > 0:
                for lookup_result in equivalent:
                    annotated_rank = df.loc[i][['rank1', 'rank2', 'rank3', 'rank4', 'rank5']].eq(lookup_result).to_numpy().nonzero()
                    if len(annotated_rank[0]) > 0: 
                        new_related_rank = annotated_rank[0][0] + 1
                        related_rank = min(related_rank, new_related_rank)

            #find related mappings to lookup value in predicted values 
            df.loc[i, 'equivalent_rank'] = min(lookup_rank, related_rank)
            
    return df

## Assing ranks to the matching 
matches are assigned their corresponding rank
non-matches are left null

In [102]:
predicted = find_ranks(model, positive_pairs,  True, True)
predicted.head(5)

Using column:  x
25/25 [==============================] - 0s 3ms/step


25/25 [==============================] - 0s 2ms/step


25/25 [==============================] - 0s 2ms/step


25/25 [==============================] - 0s 2ms/step


,x,y,Processed_x,Processed_y,Match,Distance,rank1,rank2,rank3,rank4,rank5,rank1_distance,rank2_distance,rank3_distance,rank4_distance,rank5_distance,exact_rank,equivalent_rank
1402,ALOE VERA GEL,ALOE VERA,"[1, 12, 15, 5, 27, 22, 5, 18, 1, 27, 7, 5, 12,...","[1, 12, 15, 5, 27, 22, 5, 18, 1, 0, 0, 0, 0, 0...",1,0,ALOE VERA,SLIPPERY ELM,QUERCUS ILEX,ALCEA ROSEA,ULMUS RUBRA,0.00000,0.082151,0.129918,0.138858,0.142648,1.0,1.0
60199,MEDICANL OANNAABIOS OIL VIOEE CBD,CANNABIS SATIVA,"[13, 5, 4, 9, 3, 1, 14, 12, 27, 15, 1, 14, 14,...","[3, 1, 14, 14, 1, 2, 9, 19, 27, 19, 1, 20, 9, ...",1,0,GENTIANA PNEUMONANTHE,UNCARIA TOMENTOSA,CANNABIS SATIVA,CATSCLAW,PAUSINYSTALIA JOHIMBE,0.00000,0.000000,0.092331,0.153731,0.176517,3.0,3.0
50840,GAARULIC LEMONBICAFBONATE SOA,ALLIUM SATIVUM,"[7, 1, 1, 18, 21, 12, 9, 3, 27, 12, 5, 13, 15,...","[1, 12, 12, 9, 21, 13, 27, 19, 1, 20, 9, 22, 2...",1,0,KRATOM,ARENARIA LANUGINOSA,BORAGO OFFICINALIS,LIONS TOOTH,TARAXACUM OFFICINALE,0.14579,0.164177,0.179420,0.191789,0.208097,inf,inf
4569,ASPHODELINE LUTEA,ASPHODELINE LUTEA,"[1, 19, 16, 8, 15, 4, 5, 12, 9, 14, 5, 27, 12,...","[1, 19, 16, 8, 15, 4, 5, 12, 9, 14, 5, 27, 12,...",1,0,LUDWIGIA PEPLOIDES,ASPHODELINE LUTEA,ARISTOTELIA CHILENSIS,ARISTOLOCHIA RUGOSA,PASSIFLORA EDULIS,0.00000,0.000000,0.000000,0.089544,0.111512,2.0,2.0
65816,SUPER GREJEN HZJCRN KRATOM,MITRAGYNA SPECIOSA,"[19, 21, 16, 5, 18, 27, 7, 18, 5, 10, 5, 14, 2...","[13, 9, 20, 18, 1, 7, 25, 14, 1, 27, 19, 16, 5...",1,0,MITRAGYNA SPECIOSA,KRATOM,CAMELLIA SINENSIS,SILYBUM MARIANUM,SERENOA REPENS,0.00000,0.000000,0.023898,0.101166,0.114288,1.0,1.0


----------------------------------------------------

# Predicted Match MRR Evaluation

In [103]:
models_mrr = predicted.loc[:,['x', 'exact_rank', 'equivalent_rank']]
models_mrr.loc[:,'exact_reciprocal_rank'] = 1/models_mrr.loc[:, 'exact_rank']
models_mrr.loc[:,'equivalent_reciprocal_rank'] = 1/models_mrr.loc[:, 'equivalent_rank']
models_mrr[['exact_reciprocal_rank', 'equivalent_reciprocal_rank']].describe()

,exact_reciprocal_rank,equivalent_reciprocal_rank
count,500.000000,500.000000
mean,0.537467,0.712667
std,0.347784,0.349699
min,0.000000,0.000000
25%,0.333333,0.500000
50%,0.500000,1.000000
75%,1.000000,1.000000
max,1.000000,1.000000


In [104]:
#get median and stdev
models_mrr['exact_rank'].median(), models_mrr['exact_rank'].std()

(2.0, nan)

In [105]:
#get median and stdev
models_mrr['equivalent_rank'].median(), models_mrr['equivalent_rank'].std()

(1.0, nan)

In [106]:
1/models_mrr.size  * models_mrr['exact_reciprocal_rank'].sum()

0.10749333333333334

In [107]:
1/models_mrr.size  * models_mrr['equivalent_reciprocal_rank'].sum()

0.14253333333333335

--------------------------------------

# Comparison with fuzzy string match

In [108]:
def find_Gesalt_fuzzy_lookup_and_related_rank(df: pd.DataFrame, find_related_rank: bool = False) -> pd.DataFrame:
    """For each row in the input data frame, this function utilizes the difflib implementation of fuzzy string match
       to find the top 5 unique natural product names that match the row's 'x' string value.
       
       
    Parameters
    ----------
    df: pd.DataFrame
        A pandas dataframe with the fist column containing 'x' strings to be matched against natural product names.
        Optionally containing Pandas Series with the clean encoded 'y' column.
    
    find_related_rank: bool
        A flag indicating wether to compare the top ranked results against the 'y' and it's potential equivalents or not.
      
    Returns
    -------
    df : pd.DataFrame
        Returns the padded 'x', 'y', 'rank1', 'rank2', 'rank3', 'rank4', 'rank5' seriess.
        And additionally the 'exact_rank' and 'equivalent_rank' series if requested.
    
    """
    
    df = pd.concat(
        [
            df,
            pd.DataFrame(
                df[df.columns[0]].apply(lambda x: get_close_matches(x, dUnique_df["dUnique_label"].to_list(), n=5, cutoff=0.0)).to_list(),
                columns=['rank1', 'rank2', 'rank3', 'rank4', 'rank5'],
                index = df.index
            )
        ], 
        axis=1, 
        join="inner"
    )
    
    if find_related_rank:
        df.assign(lookup_rank= np.Inf, lookup_rank_related = np.Inf)
        for i in df.index:
            # Does any of them match
            lookup_clean = clean(df.at[i , df.columns[1]])
            match_rank = df.loc[i, ['rank1', 'rank2', 'rank3', 'rank4', 'rank5']].eq(lookup_clean).to_numpy().nonzero()


            lookup_rank = np.Inf    
            if len(match_rank[0]) > 0 :
                lookup_rank = match_rank[0][0] + 1
            df.loc[i, 'exact_rank'] = lookup_rank

            equivalent = np.setdiff1d(vocab[["latin_binomial_clean","common_name_clean"]][(vocab["latin_binomial_clean"] == lookup_clean) | (vocab["common_name_clean"] == lookup_clean)].unstack().unique(), lookup_rank)

            related_rank = np.Inf
            if len(equivalent) > 0:
                for lookup_result in equivalent:
                    annotated_rank = df.loc[i][['rank1', 'rank2', 'rank3', 'rank4', 'rank5']].eq(lookup_result).to_numpy().nonzero()
                    new_related_rank = np.Inf
                    if len(annotated_rank[0]) > 0: 
                        new_related_rank = annotated_rank[0][0] + 1
                        related_rank = min(related_rank, new_related_rank)

            #find related mappings to lookup value in predicted values 
            df.loc[i, 'equivalent_rank'] = min(lookup_rank, related_rank)
    return df

In [109]:
def find_Levenshtein_fuzzy_lookup_and_related_rank(df: pd.DataFrame, find_related_rank: bool = False, report_distances: bool = False) -> pd.DataFrame:
    """For each row in the input data frame, this function utilizes the Levenshtein distance to find the 
    top 5 unique natural product names that match the row's 'x' string value.
       
       
    Parameters
    ----------
    df: pd.DataFrame
        A pandas dataframe with the fist column containing 'x' strings to be matched against natural product names.
        Optionally containing Pandas Series with the clean encoded 'y' column.
    
    find_related_rank: bool
        A flag indicating wether to compare the top ranked results against the 'y' and it's potential equivalents or not.
        
    report_distances: bool
        A flag indicating return the distance values of the top ranked results against the 'y'.
      
    Returns
    -------
    df : pd.DataFrame
        Returns the padded 'x', 'y', 'rank1', 'rank2', 'rank3', 'rank4', 'rank5' series.
        And additionally the 'exact_rank' and 'equivalent_rank' series and the 'rank1_distance', 'rank2_distance', 
        'rank3_distance', 'rank4_distance', 'rank5_distance' if requested.
    
    """
    
    print("Using column: {} as input.".format(df.columns[0]))
    
    if find_related_rank:
        print("Using column: {} as target.".format(df.columns[1]))
        df.assign(exact_rank= np.Inf, equivalent_rank = np.Inf)
        
    if report_distances:
        df.assign(rank1_distance=np.Inf, rank2_distance=np.Inf, rank3_distance=np.Inf, rank4_distance=np.Inf, rank5_distance=np.Inf)


    for i in df.index:
        distances = dUnique_df["dUnique_label"].apply(lambda x: Levenshtein.distance(x, df.at[i, df.columns[0]])).to_numpy().astype('float32')
        argsort = distances.argsort()

        
        # Top-5 smalles distances
        df.loc[i, 'rank1'] = dUnique_df['dUnique_label'][argsort[0]]  
        df.loc[i, 'rank2'] = dUnique_df['dUnique_label'][argsort[1]]
        df.loc[i, 'rank3'] = dUnique_df['dUnique_label'][argsort[2]]
        df.loc[i, 'rank4'] = dUnique_df['dUnique_label'][argsort[3]]
        df.loc[i, 'rank5'] = dUnique_df['dUnique_label'][argsort[4]]

        if report_distances:
            df.loc[i, 'rank1_distance'] = distances[argsort[0]]  
            df.loc[i, 'rank2_distance'] = distances[argsort[1]]
            df.loc[i, 'rank3_distance'] = distances[argsort[2]]
            df.loc[i, 'rank4_distance'] = distances[argsort[3]]
            df.loc[i, 'rank5_distance'] = distances[argsort[4]]

        if find_related_rank:
            df.assign(lookup_rank= np.Inf, lookup_rank_related = np.Inf)
            for i in df.index:
                # Does any of them match
                lookup_clean = clean(df.at[i , df.columns[1]])
                match_rank = df.loc[i, ['rank1', 'rank2', 'rank3', 'rank4', 'rank5']].eq(lookup_clean).to_numpy().nonzero()


                lookup_rank = np.Inf    
                if len(match_rank[0]) > 0 :
                    lookup_rank = match_rank[0][0] + 1
                df.loc[i, 'exact_rank'] = lookup_rank

                equivalent = np.setdiff1d(vocab[["latin_binomial_clean","common_name_clean"]][(vocab["latin_binomial_clean"] == lookup_clean) | (vocab["common_name_clean"] == lookup_clean)].unstack().unique(), lookup_rank)

                related_rank = np.Inf
                if len(equivalent) > 0:
                    for lookup_result in equivalent:
                        annotated_rank = df.loc[i][['rank1', 'rank2', 'rank3', 'rank4', 'rank5']].eq(lookup_result).to_numpy().nonzero()
                        new_related_rank = np.Inf
                        if len(annotated_rank[0]) > 0: 
                            new_related_rank = annotated_rank[0][0] + 1
                            related_rank = min(related_rank, new_related_rank)

                #find related mappings to lookup value in predicted values 
                df.loc[i, 'equivalent_rank'] = min(lookup_rank, related_rank)

    return df

--------------------------------------

# Fuzzy Levenshtein Match MRR Evaluation

In [110]:
Levenshtein_match = find_Levenshtein_fuzzy_lookup_and_related_rank(positive_pairs.loc[:, ('x', 'y')], True, True)

Using column: x as input.
Using column: y as target.


In [111]:
Levenshtein_mrr = Levenshtein_match.loc[:,['x', 'exact_rank', 'equivalent_rank']]
Levenshtein_mrr.loc[:,'exact_reciprocal_rank'] = 1/Levenshtein_mrr.loc[:, 'exact_rank']
Levenshtein_mrr.loc[:,'equivalent_reciprocal_rank'] = 1/Levenshtein_mrr.loc[:, 'equivalent_rank']
Levenshtein_mrr[['exact_reciprocal_rank', 'equivalent_reciprocal_rank']].describe()

,exact_reciprocal_rank,equivalent_reciprocal_rank
count,500.000000,500.000000
mean,0.421867,0.675433
std,0.476650,0.446462
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,1.000000
75%,1.000000,1.000000
max,1.000000,1.000000


In [112]:
#get median and stdev
Levenshtein_mrr['exact_rank'].median(), Levenshtein_mrr['exact_rank'].std()

(inf, nan)

In [113]:
#get median and stdev
Levenshtein_mrr['equivalent_rank'].median(), Levenshtein_mrr['equivalent_rank'].std()

(1.0, nan)

In [114]:
1/Levenshtein_mrr.size  * Levenshtein_mrr['exact_reciprocal_rank'].sum()

0.08437333333333334

In [115]:
1/Levenshtein_mrr.size  * Levenshtein_mrr['equivalent_reciprocal_rank'].sum()

0.1350866666666667

--------------------------------------

# Fuzzy Gesalt Match MRR Evaluation

In [116]:
fuzzy_match = find_Gesalt_fuzzy_lookup_and_related_rank(positive_pairs.loc[:, ('x', 'y')], True)

In [117]:
fuzzy_mrr = fuzzy_match.loc[:,['x', 'exact_rank', 'equivalent_rank']]
fuzzy_mrr.loc[:,'exact_reciprocal_rank'] = 1/fuzzy_mrr.loc[:, 'exact_rank']
fuzzy_mrr.loc[:,'equivalent_reciprocal_rank'] = 1/fuzzy_mrr.loc[:, 'equivalent_rank']
fuzzy_mrr[['exact_reciprocal_rank', 'equivalent_reciprocal_rank']].describe()

,exact_reciprocal_rank,equivalent_reciprocal_rank
count,500.000000,500.000000
mean,0.452600,0.736533
std,0.480036,0.416172
min,0.000000,0.000000
25%,0.000000,0.333333
50%,0.200000,1.000000
75%,1.000000,1.000000
max,1.000000,1.000000


In [118]:
#get median and stdev
fuzzy_mrr['exact_rank'].median(), fuzzy_mrr['exact_rank'].std()

(5.0, nan)

In [119]:
#get median and stdev
fuzzy_mrr['equivalent_rank'].median(), fuzzy_mrr['equivalent_rank'].std()

(1.0, nan)

In [120]:
1/fuzzy_mrr.size  * fuzzy_mrr['exact_reciprocal_rank'].sum()

0.09052

In [121]:
1/fuzzy_mrr.size  * fuzzy_mrr['equivalent_reciprocal_rank'].sum()

0.14730666666666667

--------------------------------------

# Translation tests

In [122]:
translation = '../data/translation_test_nps_202203171038.csv'
translation_set = preprocessInput(translation, MAXLEN)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5950 entries, 0 to 5949
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   np_name  5950 non-null   object
dtypes: object(1)
memory usage: 46.6+ KB
None
Processing file: ----------------------------------------
Renaming colums:
	Current names: Index(['np_name'], dtype='object')
	New names: Index(['x'], dtype='object')
Dropping row with empty cells:
	Dropped 0 rows with empty cells.
Dropping sequences longer than the maxlen of 65:
	Dropped 1 that exceeded the maximum sequence length.
	Cleaning string sequences.
	Uppercasing string sequences.
Dropping duplicate sequences:
	Dropped 35 duplicate sequences.
Done processing: ---------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5914 entries, 0 to 5913
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   x       5914 non-null   object
dtypes: o

In [123]:
translation_set.head()

,x
0,XTNNP
1,NWLMKV
2,WZMFCEA
3,AARONSROD
4,AARONS ROD WHOLE


In [124]:
encode_pad_tag(translation_set, 1, 0, MAXLEN)

Encoding and Padding: ----------------------------------
	Processing x
Tagging: -----------------------------------------------


,x,Processed_x,Match,Distance
0,XTNNP,"[24, 20, 14, 14, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0...",1,65
1,NWLMKV,"[14, 23, 12, 13, 11, 22, 0, 0, 0, 0, 0, 0, 0, ...",1,65
2,WZMFCEA,"[23, 26, 13, 6, 3, 5, 1, 0, 0, 0, 0, 0, 0, 0, ...",1,65
3,AARONSROD,"[1, 1, 18, 15, 14, 19, 18, 15, 4, 0, 0, 0, 0, ...",1,65
4,AARONS ROD WHOLE,"[1, 1, 18, 15, 14, 19, 27, 18, 15, 4, 27, 23, ...",1,65
...,...,...,...,...
5909,ZINGIBER ZINGIBER WHOLE,"[26, 9, 14, 7, 9, 2, 5, 18, 27, 26, 9, 14, 7, ...",1,65
5910,ZYGAENA ERYTHRAEA WHOLE,"[26, 25, 7, 1, 5, 14, 1, 27, 5, 18, 25, 20, 8,...",1,65
5911,ZYGAENA INDICA WHOLE,"[26, 25, 7, 1, 5, 14, 1, 27, 9, 14, 4, 9, 3, 1...",1,65
5912,ZYGAENA LEWINI WHOLE,"[26, 25, 7, 1, 5, 14, 1, 27, 12, 5, 23, 9, 14,...",1,65


In [125]:
novelty = find_ranks(model, translation_set.sample(n=10), False)

Using column:  x
25/25 [==============================] - 0s 2ms/step


In [126]:
novelty.head(10)

,x,Processed_x,Match,Distance,rank1,rank2,rank3,rank4,rank5
2657,GENERAL PLANTAIN WHOLE,"[7, 5, 14, 5, 18, 1, 12, 27, 16, 12, 1, 14, 20...",1,65,GOLDENSEAL,COLEUS,CORDYCEPS,COPTIS,LICORICE
1281,CAPSELLA SUBBERGIANA WHOLE,"[3, 1, 16, 19, 5, 12, 12, 1, 27, 19, 21, 2, 2,...",1,65,GOLDENSEAL,SENNA,COPTIS,COLEUS,CORDYCEPS
2619,GALIUM LARECAJENSE WHOLE,"[7, 1, 12, 9, 21, 13, 27, 12, 1, 18, 5, 3, 1, ...",1,65,CORDYCEPS,GOLDENSEAL,OLIVE LEAF,GUGGUL,HORNY GOAT WEED
3484,MAHONIA AQUIFOLIUM,"[13, 1, 8, 15, 14, 9, 1, 27, 1, 17, 21, 9, 6, ...",1,65,ZANTHOXYLUM OVALIFOLIUM,GALIUM UNIFLORUM,ADIANTUM LATIFOLIUM,DIPSACUS FULLONUM,CHAMOMILE
5090,SOLANUM YANGAMBIENSE WHOLE,"[19, 15, 12, 1, 14, 21, 13, 27, 25, 1, 14, 7, ...",1,65,SENNA,COLEUS,CORDYCEPS,GOLDENSEAL,BLACK PEPPER
1312,CARDUUS MARIANUS WHOLE,"[3, 1, 18, 4, 21, 21, 19, 27, 13, 1, 18, 9, 1,...",1,65,GOLDENSEAL,COPTIS,CORDYCEPS,BARBERRY,CITRUS AURANTIUM
1583,CINNAMOMUM CAMPHOROIDES WHOLE,"[3, 9, 14, 14, 1, 13, 15, 13, 21, 13, 27, 3, 1...",1,65,CINNAMOMUM VERUM,CINNAMOMUM CAMPHORA,GOLDENSEAL,BLACK PEPPER,COLEUS
3306,LESSER GALANGAL WHOLE,"[12, 5, 19, 19, 5, 18, 27, 7, 1, 12, 1, 14, 7,...",1,65,BLACK CHERRY,COLEUS,GOJI BERRY,CORDYCEPS,BLACK PEPPER
1709,CITRUS X RACEMOSA WHOLE,"[3, 9, 20, 18, 21, 19, 27, 24, 27, 18, 1, 3, 5...",1,65,COPTIS,GOLDENSEAL,CORDYCEPS,CISTUS CRETICUS,GOJI BERRY
4542,RHEUMATISM WHOLE,"[18, 8, 5, 21, 13, 1, 20, 9, 19, 13, 27, 23, 8...",1,65,GUGGUL,HORNY GOAT WEED,COLEUS,SILYBUM MARIANUM,COMMIPHORA MUKUL


In [127]:
# test["Processed_np_name"] = test.np_name.apply(clean).apply(encode).apply(padding)

In [128]:
# test = preprocessInput(unmapped, converters = {"drug_name_original":str}, skip_blank_lines=True, na_filter=True, na_values="")
# test["Processed_drug_name_original"] = test[test.columns[0]].apply(clean).apply(encode).apply(padding)